# Data Inconsistencies

In [1]:
import pandas as pd
from siuba import *

In [2]:
from dla_utils import _dla_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/dla_df.parquet")

In [4]:
df.sample()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
20891,Obligated,HSIPL,5007(088),Santa Barbara,2022-02-03,2022-02-08,2022-02-17,2022-02-17,2022-02-23,21894,...,0.00,Large,1,0,0,0,0,0,0,1


**NOTE** this notebook was made before offical `primary_agency_name` and `locode` was known. `agency` is used to show the inconsistencies that arise when looking at certain `project_ids` and `locode`

## Looking into MPOs

using this map to see where the agency is in realtion to nearest mpo https://gisdata-caltrans.opendata.arcgis.com/datasets/b3e0ef95520843ba8c1d3b9c0fa9a607_0/explore

In [5]:
print(len(df>>count(_.mpo)>>arrange(-_.n)))

35


* Accoring to the Caltrans GIS data, there are 18 mpos

In [6]:
#using _dla_utils
(_dla_utils.find_top(df))>>filter(_.variable=='mpo')

,value,count,variable
0,SCAG,6109,mpo
1,MTC,3955,mpo
2,NON-MPO,3288,mpo
3,SACOG,2216,mpo
4,CFCG,1589,mpo
5,KCOG,1169,mpo
6,AMBAG,1022,mpo
7,STANCOG,943,mpo
8,TCAG,590,mpo
9,SJCG,583,mpo


### Which MPOs have the least amount of obligations? What Agencies do they include?

In [7]:
df >> count(_.mpo)>>arrange(-_.n)>> filter(_.n==1)

,mpo,n
6,FSTIP,1
11,MNOLTC,1
15,SANDA,1


#### MPO SANDA

In [8]:
df>>filter(_.mpo =='SANDA')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
12809,Obligated,HSIPLN,5426(008),Big Bear Lake,2013-12-09,2014-01-27,2014-02-06,2014-02-06,2014-02-07,169650,...,0.00,Small,0,0,0,1,0,1,0,2


* maybe meant "SANDAG" but Big Bear Lake is in "SCAG" jurisdiction
* looked to excel download file and this there is anothe Big Bear Lake and SANDAG instance

In [9]:
df>>filter(_.agency.str.contains('Big Bear')) >> filter(_.mpo.str.contains("SANDA"))

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
12609,Obligated,HSIPLN,5426(011),Big Bear Lake,2014-02-20,2014-02-21,2014-03-02,2014-03-03,2014-03-04,27000,...,0.00,Small,0,0,0,0,0,1,0,1
12809,Obligated,HSIPLN,5426(008),Big Bear Lake,2013-12-09,2014-01-27,2014-02-06,2014-02-06,2014-02-07,169650,...,0.00,Small,0,0,0,1,0,1,0,2


google maps show the `project_location` intersections in the jurisdiction of Big Bear and not in SANDAG

#### MPO SHASTA

In [10]:
df >>filter(_.mpo=='SHASTA')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10079,Obligated,HSIPL,5073(075),Orange,2014-12-30,2015-01-08,2015-01-16,2015-01-16,2015-01-18,196199,...,0.00,Large,0,0,0,1,0,0,0,1
23300,Obligated,ER,32L0(433),Shasta County,2023-02-02,2023-02-02,2023-02-02,2023-02-02,2023-02-09,-8084,...,0.00,Large,0,0,0,1,0,1,0,2
23351,Obligated,BRLS,5068(035),Redding,2023-02-02,2023-02-02,2023-02-05,2023-02-08,2023-02-15,0,...,0.00,Large,0,0,1,0,0,1,0,2
23352,Obligated,BRLS,5068(036),Redding,2022-12-23,2023-02-01,2023-02-14,2023-02-16,2023-02-23,867593,...,0.00,Large,0,0,1,0,0,1,0,2
23353,Obligated,BRLS,5068(038),Redding,2023-02-02,2023-02-02,2023-02-05,2023-02-08,2023-02-15,0,...,0.00,Large,0,0,1,0,0,1,0,2
23539,Obligated,BRLS,5906(124),Shasta County,2023-02-24,2023-02-24,2023-03-03,2023-03-07,2023-03-15,26559,...,0.00,Large,0,0,1,0,0,1,0,2
23741,Obligated,BRLS,5068(001),Redding,2023-03-29,2023-03-29,2023-03-29,2023-03-29,2023-04-04,0,...,0.00,Large,0,0,1,0,0,1,0,2
23899,Obligated,BRLO,5906(087),Shasta County,2023-03-29,2023-04-18,2023-04-21,2023-04-24,2023-05-02,10000,...,0.00,Large,0,0,1,0,0,1,0,2
23901,Obligated,BRLS,5906(107),Shasta County,2023-05-08,2023-05-08,2023-05-11,2023-05-12,2023-05-17,0,...,0.00,Large,0,0,1,0,0,1,0,2
23904,Obligated,BRLS,5068(001),Redding,2023-05-08,2023-05-08,2023-05-08,2023-05-08,2023-05-12,-1191706,...,0.00,Large,0,0,1,0,0,1,0,2


In [11]:
df>> filter(_.agency.str.contains('Shasta')) >> count(_.mpo)

,mpo,n
0,MTC,3
1,SACOG,4
2,SHASTA,10
3,STPA,142


In [12]:
df>>filter(_.agency.str.contains('Shasta')) >> filter(_.mpo=="MTC")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
19507,Obligated,ACSTP,15A6(002),Shasta County,2021-05-18,2021-05-28,2021-06-02,2021-07-12,2021-07-22,0,...,"19,251.02",Large,0,0,0,0,0,1,0,1
21035,Obligated,ER,15A6(002),Shasta County,2022-02-22,2022-02-22,2022-02-22,2022-02-25,2022-03-04,19251,...,"-17,824.55",Large,0,0,0,0,0,1,0,1
22987,Obligated,ACSTER,15A6(002),Shasta County,2022-09-23,2022-09-23,2022-10-10,2022-12-05,2022-12-12,0,...,"299,854.53",Large,0,0,0,0,0,1,0,1


* in the project location column, the intersection is Tustin and Mayfair which is in Orange Couny, or SCAG. Agency is also Orange, yet the mpo is Shasta

#### MPO MNOLTC and Agencies under

In [13]:
df>> filter(_.mpo=='MNOLTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10862,Obligated,RPSTPL,5205(019),Rialto,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2014-09-10,2000000,...,"-2,289,215.19",Medium,1,0,0,0,0,0,0,1


In [14]:
#looking to see if what other MPOs Rialto is in, should be in SCAG
df>> filter(_.agency=='Rialto') >> count(_.mpo)

,mpo,n
0,KCOG,1
1,MNOLTC,1
2,SANDAG,1
3,SCAG,18


In [15]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='KCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10709,Obligated,SRTSL,5205(014),Rialto,2014-10-08,2014-10-08,2014-10-08,2014-10-08,2014-10-14,0,...,0.00,Medium,0,0,0,1,0,0,0,1


In [16]:
df>> filter(_.agency=='Rialto') >> filter(_.mpo=='SANDAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
12458,Obligated,HSIPL,5205(017),Rialto,2014-03-07,2014-03-07,2014-03-21,2014-03-21,2014-03-21,-89313,...,0.00,Medium,1,0,0,1,0,1,0,3


#### FSTIP

In [17]:
df>>filter(_.mpo=='FSTIP')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
12053,Obligated,CML,5160(023),Los Banos,2014-04-16,2014-04-16,2014-04-24,2014-04-24,2014-04-24,-16940,...,0.00,Medium,1,0,0,0,0,0,0,1


* FSTIP is the Federal Statewide Transportation Improvement Program. 

### Number of Unique Agencies in Each MPO

In [18]:
#using _dla_utils
_dla_utils.get_nunique(df, "primary_agency_name", "mpo")

,mpo,n
19,SCAG,263
12,MTC,149
13,NON-MPO,126
28,STANCOG,91
16,SANDAG,82
24,SJCOG,77
9,MCAG,50
8,KCOG,45
14,SACOG,38
3,COFCG,27


In [19]:
#using _dla_utils
(_dla_utils.find_top((df>>filter(_.mpo=='SCAG'))))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Los Angeles,571,primary_agency_name
1,Los Angeles County,518,primary_agency_name
2,San Bernardino County,163,primary_agency_name
3,Riverside County,145,primary_agency_name
4,Ventura County,127,primary_agency_name
5,Long Beach,106,primary_agency_name
6,Orange County Transportation Authority,105,primary_agency_name
7,Santa Ana,105,primary_agency_name
8,Los Angeles County Metropolitan Transportation...,98,primary_agency_name
9,Imperial County,97,primary_agency_name


## Function for Agencies in each MPO

creating a function to look into each MPO. What agencies are listed under the MPO name? Are the projects located in that jurisdiction? 

In [20]:
_dla_utils.interactive_widget_counts(df, 'mpo', 'primary_agency_name')

Dropdown(description='Mpo', options=('AMBAG', 'BCAG', 'CFCG', 'COFCG', 'ER NONE', 'FCOG', 'FSTIP', 'KCAG', 'KC…

Output()

### Filter to see values in column 

In [21]:
#using Siuba filter, we can start to look at the agenies under different MPOs

In [22]:
df>>filter(_.agency=='Marin County')>> filter(_.mpo=='SCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
19910,Obligated,ACSTP,40A0(087),Marin County,2021-06-16,2021-06-17,2021-06-17,2021-09-13,2021-09-14,0,...,"676,463.58",Large,0,0,0,1,0,0,0,1
20874,Obligated,ER,40A0(087),Marin County,2022-02-16,2022-02-16,2022-02-16,2022-02-22,2022-02-24,754476,...,"-698,571.41",Large,0,0,0,1,0,0,0,1


* projet located in Lucas Valley, which is not in SCAG

In [23]:
df>>filter(_.agency=='Sonora')>> count(_.mpo)

,mpo,n
0,COFCG,1
1,NON-MPO,6
2,SCAG,2
3,STANCOG,1


In [24]:
df>>filter(_.agency=='Sonora')>> filter(_.mpo=='SCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
8383,Obligated,CML,5010(006),Sonora,2015-07-23,2015-08-20,2015-08-26,2015-08-26,2015-08-27,0,...,"46,068.46",Small,1,1,0,0,0,0,0,2
8679,Obligated,CML,5010(008),Sonora,2015-07-08,2015-07-08,2015-07-15,2015-07-17,2015-07-21,205000,...,0.00,Small,0,0,0,0,0,0,0,0


In [25]:
df>>filter(_.agency=='Shasta County')>> filter(_.mpo=='MTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
19507,Obligated,ACSTP,15A6(002),Shasta County,2021-05-18,2021-05-28,2021-06-02,2021-07-12,2021-07-22,0,...,"19,251.02",Large,0,0,0,0,0,1,0,1
21035,Obligated,ER,15A6(002),Shasta County,2022-02-22,2022-02-22,2022-02-22,2022-02-25,2022-03-04,19251,...,"-17,824.55",Large,0,0,0,0,0,1,0,1
22987,Obligated,ACSTER,15A6(002),Shasta County,2022-09-23,2022-09-23,2022-10-10,2022-12-05,2022-12-12,0,...,"299,854.53",Large,0,0,0,0,0,1,0,1


In [26]:
df>>filter(_.agency=='Marina')>> filter(_.mpo=='AMBAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
4849,Obligated,RPSTPL,5416(008),Marina,2017-02-28,2017-02-28,2017-02-28,2017-02-28,2017-03-10,-13679,...,0.00,Small,1,0,0,0,0,0,0,1
5514,Obligated,RPSTPL,5416(008),Marina,2016-11-01,2016-11-01,2016-11-01,2016-11-01,2016-11-07,55689,...,0.00,Small,1,0,0,0,0,0,0,1
5585,Obligated,RPSTPL,5416(008),Marina,2016-07-27,2016-10-25,2016-10-25,2016-10-25,2016-10-28,-55689,...,0.00,Small,1,0,0,0,0,0,0,1
6816,Obligated,HSIPL,5416(010),Marina,2016-05-05,2016-05-05,2016-05-06,2016-05-06,2016-05-13,1300033,...,0.00,Small,0,0,0,1,0,0,0,1
9170,Obligated,RPSTPL,5416(011),Marina,2015-05-21,2015-05-21,2015-05-21,2015-05-22,2015-05-27,1650000,...,0.00,Small,0,0,0,1,0,0,0,1
9304,Obligated,RPSTPL,5416(008),Marina,2015-05-05,2015-05-05,2015-05-05,2015-05-05,2015-05-06,2000000,...,"-2,286,502.66",Small,1,0,0,0,0,0,0,1
12808,Obligated,RPSTPL,5416(008),Marina,2014-02-07,2014-02-07,2014-02-07,2014-02-07,2014-02-07,0,...,0.00,Small,1,0,0,0,0,0,0,1
12866,Obligated,RPSTPL,5416(008),Marina,2014-01-09,2014-01-23,2014-01-28,2014-01-28,2014-01-28,0,...,"2,289,215.19",Small,1,0,0,0,0,0,0,1
14995,Obligated,HSIPL,5416(010),Marina,2019-10-08,2019-10-08,2019-10-08,2019-10-08,2019-10-21,-19,...,0.00,Small,0,0,0,1,0,0,0,1


In [27]:
df>>filter(_.agency=='Kern County (District 9)')>> filter(_.mpo=='BCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
2481,Obligated,CML,5961(006),Kern County (District 9),2018-02-05,2018-02-08,2018-02-13,2018-02-13,2018-02-21,1327950,...,0.00,Large,0,0,0,0,0,1,0,1


In [28]:
df>>filter(_.agency=='Santa Barbara County')>> filter(_.mpo=='BCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
293,Obligated,ER,38F0(011),Santa Barbara County,2018-09-06,2018-09-06,2018-10-29,2018-10-31,2018-11-06,83500,...,0.00,Large,0,0,0,1,0,0,0,1
1041,Obligated,ER,38F0(011),Santa Barbara County,2018-07-05,2018-07-05,2018-07-05,2018-07-05,2018-07-13,15000,...,"-16,186.54",Large,0,0,0,1,0,0,0,1


In [29]:
df>>filter(_.agency=='Bishop')>> filter(_.mpo=='KCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10707,Obligated,RPSTPL,5125(021),Bishop,2014-09-23,2014-10-08,2014-10-08,2014-10-08,2014-10-14,4049,...,0.00,Small,1,0,0,0,0,0,0,1


In [30]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='MCAG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10873,Obligated,RPSTPL,5953(685),Los Angeles,2014-09-05,2014-09-05,2014-09-08,2014-09-10,2014-09-10,378000,...,"-432,661.67",Large,1,0,0,0,0,0,0,1
10916,Obligated,BRLSZD,5006(337),Los Angeles,2014-08-20,2014-08-20,2014-08-26,2014-08-29,2014-08-29,58000,...,0.00,Large,0,0,0,0,0,1,0,1
11446,Obligated,HSIPL,5006(551),Los Angeles,2014-07-01,2014-07-01,2014-07-08,2014-07-09,2014-07-09,0,...,0.00,Large,0,0,0,1,0,0,0,1
11713,Obligated,CML,5006(647),Los Angeles,2014-06-10,2014-06-10,2014-06-10,2014-06-17,2014-06-18,0,...,0.00,Large,1,0,0,0,0,0,0,1
11754,Obligated,STPLZ,5953(564),Los Angeles,2014-04-28,2014-05-23,2014-06-02,2014-06-03,2014-06-12,767290,...,0.00,Large,0,0,0,0,0,1,0,1
11873,Obligated,BRLSZD,5953(566),Los Angeles,2013-08-28,2014-05-06,2014-05-13,2014-05-20,2014-05-22,89563,...,0.00,Large,0,0,1,0,0,1,0,2


In [31]:
df>>filter(_.agency=='Los Angeles')>> count(_.mpo)

,mpo,n
0,ER NONE,1
1,KCOG,4
2,MCAG,6
3,NON-MPO,12
4,SANDAG,14
5,SCAG,598
6,SJCOG,4
7,STANCOG,3
8,NaN,3


In [32]:
df>>filter(_.agency=='Indio')>> filter(_.mpo=='MCTC')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
8026,Obligated,CML,5275(022),Indio,2015-10-08,2015-10-12,2015-11-10,2015-11-10,2015-11-15,81189,...,0.00,Large,0,0,0,1,0,1,0,2


In [33]:
df>>filter(_.agency=='Los Angeles')>> filter(_.mpo=='SJCOG')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
11054,Obligated,RPSTPL,5006(716),Los Angeles,2014-08-04,2014-08-04,2014-08-07,2014-08-13,2014-08-13,669000,...,"-765,742.48",Large,0,0,0,1,0,0,0,1
11678,Obligated,BHLS,5006(193),Los Angeles,2014-03-28,2014-06-09,2014-06-13,2014-06-18,2014-06-19,710400,...,0.00,Large,0,0,1,0,0,1,0,2
11734,Obligated,STPL,5006(469),Los Angeles,2014-06-10,2014-06-10,2014-06-10,2014-06-12,2014-06-13,-39022,...,0.00,Large,0,0,0,0,0,0,0,0
11843,Obligated,HPLUL,5006(518),Los Angeles,2014-04-28,2014-05-09,2014-05-22,2014-05-23,2014-05-29,-127527,...,0.00,Large,0,0,0,1,0,1,0,2


### Findings from Filtering

### MPOs with all correct agencies

* AMBAG
* KCAG
* SBCG
* TMPO

### MPOs with some misplaced agencies

* BCAG
* MTC
* SCAG
* KGOC
* MNOLTC
* FCOG 
* KCOG
* MCAG
* SJCOG
* STANCOG
* MCTC
* SACOG
* SJCOG
* STANCOG 
* TCAG


### MPOs not defined / all incorrect  
* CFCG - has Fresno Counties. Fresno is part of FCOG
* COFCG - combination of multiple MPOs
* FCOG - includes Ventura and Fresno
* FSTIP - a Prefix rather than a MPO
* MNOLTC - one entry
* SANDA - one entry
* SDAG - maybe SANDAG
* SANDAG - mostly SCAG Agencies
* SHASTA - one agency not in Shasta County
* SJCG - most likely SJCOG
* SLAC - maybe SLOCOG
* STACOG - STANCOG
* STNCOG - STANCOG
* STPA - maybe SRTA


## Agencies
Most of these agencies are corrected in the [change_agency_to_locodelist](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/change_agency_to_locodelist.ipynb)

In [34]:
df.agency.nunique()

686

In [35]:
df.primary_agency_name.nunique()

619

In [36]:
sorted(df.agency.unique())

['Access Services',
 'Acsr',
 'Agoura Hills',
 'Alameda',
 'Alameda - Contra Costa Transit District',
 'Alameda Corridor Transportation Authority',
 'Alameda County',
 'Alameda County Congestion Management Agency',
 'Alameda County Transportation Commission',
 'Alameda County Transportation Improvement Authority',
 'Alameda County Waste Management Authority',
 'Albany',
 'Alhambra',
 'Alpine County',
 'Amador',
 'Amador City',
 'Amador County',
 'Amador County Transportation Commission',
 'Amador Rapid Transit System',
 'American Canyon',
 'Anaheim',
 'Angels',
 'Antelope Valley Transit Authority',
 'Antioch',
 'Apple Valley',
 'Arcadia',
 'Arcata',
 'Arroyo Grande',
 'Artesia',
 'Arvin',
 'Association Of Monterey Bay Area Governments',
 'Atascadero',
 'Atherton',
 'Atwater',
 'Auburn',
 'Avenal',
 'Azusa',
 'Bakersfield',
 'Baldwin Park',
 'Banning',
 'Barstow',
 'Bay Area Air Quality Management District',
 'Beaumont',
 'Bell',
 'Bell Gardens',
 'Bellflower',
 'Belmont',
 'Benicia',
 

In [37]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.agency.str.contains("Amador"))), 'project_no','agency')

,agency,n
2,Amador County,12
0,Amador,4
5,Livermore Amador Valley Transit Authority,4
3,Amador County Transportation Commission,2
1,Amador City,1
4,Amador Rapid Transit System,1


* list of agencies to be corrected with incorrect locodes
    * includes the null locodes

|Project No/Correct Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |
| 5926 | Amador | Amador County |
| 6128 | Calaveras C T C | Calaveras Council of Governments |
| 5930 | Calaveras | Calaveras County |
| 6498 | Cap Soeast Con | Capital Southeast Connector |
| 6164 | Coachella | Coachella Valley Association Of Governments |
| 5384 | Dst Hot Springs | Desert Hot Springs |
| 6061 | Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5230 | Gustine City | Gustine |
| 5387 | Hawaiian Garden | Hawaiian Gardens |
| 5958 | Imperial | Imperial County |
| 4825 | Imperial County | 5958 |
| 5948 | Inyo | Inyo County |
| 5940 | Mariposa | Mariposa County |
| 40A0 | Mendocino County | 5910 |
| 5939 | Merced | Merced County |
| 0484 | Merced County | Merced County |
| 5947 | Mono | Mono County |
| 22X0 | Portola Valley | 5390 |
| 5420 | Ranchocucamonga | Rancho Cucamonga |
| 5413 | Ranchopalosverd | Rancho Palos Verdes |
| 5956 | Riverside | Riverside County |
| 5349 | Rollinghillsest | Rolling Hills Estates |
| 5954 | San Bernardino | San Bernardino  County |
| 5957 | San Diego | San Diego County |
| 5929 | San Joaquin | San Joaquin County |
| 6088 | San Joaquin Cog | San Joaquin Council Of Governments |
| 5340 | Santa Fe Sprgs | Santa Fe Springs |
| 6066 | Sd Assoc Gov’T | San Diego Association of Governments |
| 20W0 | Sierra Madre | 5158 |
| 6503 | Solano County Transit | 6497 |
| 5938 | Stanislaus | Stanislaus County |
| 5932 | Tuolumne | Tuolumne County |
| 5448 | Twentynine Palm | Twentynine Palms |
| 5952 | Ventura | Ventura County |
| 5026 | San Buena Ventura | San Buenaventura |
| 5338 | Westminister | Westminster |

### Acronym Agencies Totals

In [38]:
#https://stackoverflow.com/questions/19937362/python-pandas-filter-string-data-based-on-its-string-length
df_acronym = df[df['agency'].apply(lambda x: len(x)<5)]

In [39]:
df_acronym>>count(_.agency, _.locode)

,agency,locode,n
0,Acsr,6312,11
1,Bell,5272,12
2,Brea,5237,11
3,Brea,nan,5
4,Dwr,nan,3
5,Galt,5293,13
6,Inyo,nan,24
7,Ione,5311,2
8,Jpam,nan,3
9,Kern,5961,2


In [40]:
df_acronym>>filter(_.agency=='Sgvc')>>count(_.project_no)

,project_no,n
0,6303(020),2
1,6303(026),1
2,6303(028),1
3,6303(030),1
4,7500(286),2


* only one acronym with a locode. checking agencies with no locode

In [41]:
# using _dla_utils
_dla_utils.get_nunique(df_acronym, 'project_no', 'agency')

,agency,n
5,Inyo,18
21,Taft,15
13,Napa,10
2,Brea,8
10,Lodi,8
4,Galt,7
14,Ojai,7
15,Ross,7
18,Sgvc,5
1,Bell,4


* 9 agencies that have acronyms when not including locodes

In [42]:
df_acronym>>filter(_.locode.isnull())>>count(_.agency)

,agency,n


So the agency 'Sgvc' uses two 'locodes' in the `project_no` column, one of which '7500' we know is used with multiple agencies as seen in the [add_locodes](https://github.com/cal-itp/data-analyses/blob/main/dla/e76_obligated_funds/add_locodes.ipynb) notebook.

### **Agencies with Acronyms and NaN Locodes:**

In [43]:
print(len(df>>filter(_.locode.isnull())))

0


In [44]:
# number of entries with no Locode- around ~7.5%

In [45]:
print(len(df>>filter(_.locode.isnull())>>count(_.agency)))

0


In [46]:
#number of agencies with Locodes missing around ~31% 

In [47]:
df>>filter(_.locode.isnull())>>count(_.agency)>>arrange(-_.n)

,agency,n



* using the most common first four numbers of the `project_no`


|Project No/Corresponding Locode |Agency in Obligated List | Actual Name (in Locode List) | 
| --------------- | --------------- | --------------- |
| 6053 | Sanbag  | San Bernardino Associated Governments | 
| 6066 | Sd Assoc Gov'T  | San Diego Association of Governments |
| 6071 | Ora Co Trans Au  | Orange County Transportation Authority |
| 6089 | Stan Area Aog  | Stanislaus Council of Governments |
| 5181 | Huntington Beac  | Huntington Beach |
| 6054 | Riv Co Trans Co | Riverside County Transportation Commission |
| 6128 |  Calaveras C T C | Calaveras Council of Governments |
| 5417 |  La Canada-Flint | La Canada Flintridge |
| 6303 | Sgvc | San Gabriel Valley Council of Governments |
| 6349 |  Stpd | Stockton Port District |
| 6305 |  Yrts | Yosemite Area Regional Transportation System JPA |
| 6248 |  Dwr | Department of Water Resources |
| 6308 |  Jpam | Transit Joint Powers Authority For Merced County |
| 6088 |  San Joaquin Cog  | San Joaquin Council of Governments |
| 5026 |  Sanbuenaventura | San Buenaventura |
| 5372 | Sanjuancapistra | San Juan Capistrano  |
| 5478 |  Rsm | Rancho Santa Margarita |
| 6190 |  Usda Forest Ser | U.S. Forest Service, Pacific Southwest Region |
| 6498 |  Cap Soeast Con  | Capital Southeast Connector |
| 6164 | Coachella V A G |  Coachella Valley Association of Governments |
| 6061 |  Fthll/East Tca | Foothill Eastern Transportation Corridor |
| 5476 |  Lgnw |  Laguna Woods |
| 6095 | Merced Co Aog | Merced County Association of Governments |
| 5413 |  Ranchopalosverd | Rancho Palos Verdes |
| 5349 |  Rollinghillsest | Rolling Hills Estates |
| 6373 |  Samd | South Coast Air Quality Management District |
| 6245 |  Sjrt |  San Joaquin Regional Transit District |


In [48]:
#printing number of entries for Westmorland which has an offical locode of 5278 
print(len(df>>filter(_.locode=="5278")))
print(len(df>>filter(_.agency=="Westmorland")))

13
14


In [49]:
df>>filter(_.agency=="Westmorland")>>count(_.agency)

,agency,n
0,Westmorland,14


In [50]:
df>>filter(_.agency.str.contains("Huntington Beac"))>>count(_.agency)

,agency,n
0,Huntington Beac,6
1,Huntington Beach,58


#### Dwr

In [51]:
df>>filter(_.agency=="Dwr")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
8995,Obligated,BRLOZ,6248(011),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-93034,...,0.00,Large,0,0,0,0,0,1,0,1
8996,Obligated,BRLOZ,6248(013),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-96147,...,0.00,Large,0,0,0,0,0,1,0,1
8997,Obligated,BHLOZ,6248(030),Dwr,2015-06-12,2015-06-12,2015-06-12,2015-06-12,2015-06-15,-128952,...,0.00,Large,0,0,0,0,0,1,0,1


* locode list shows this as "Department of Water Resources" 

#### Jpam

In [52]:
df>>filter(_.agency=="Jpam")

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
9508,Obligated,CML,6308(016),Jpam,2015-03-23,2015-03-23,2015-04-03,2015-04-06,2015-04-07,100000,...,0.00,Medium,0,1,0,0,0,0,0,1
10285,Obligated,CML,6308(013),Jpam,2014-12-26,2014-12-26,2014-12-26,2014-12-26,2014-12-30,-183102,...,0.00,Medium,0,1,0,0,0,0,0,1
11793,Obligated,CML,6308(014),Jpam,2014-05-20,2014-05-21,2014-06-03,2014-06-06,2014-06-06,100000,...,0.00,Medium,0,1,0,0,0,0,0,1


* Showing in locode list as "Transit Joint Powers Authority For Merced County"

In [53]:
df>>filter(_.project_location=='Stanislaus County')>>count(_.agency)

,agency,n
0,Stan Area Aog,3
1,Stanislaus,2
2,Stanislaus Council Of Governments,6
3,Stanislaus County,3


In [54]:
df>>filter(_.project_location=='Stanislaus County')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
680,Obligated,CML,6089(055),Stanislaus Council Of Governments,2018-08-28,2018-08-28,2018-08-28,2018-08-29,2018-09-04,0,...,0.00,Large,0,0,0,0,0,0,1,1
5228,Obligated,CML,6089(062),Stanislaus Council Of Governments,2016-12-22,2016-12-22,2016-12-22,2016-12-22,2016-12-23,-4000,...,0.00,Large,0,0,0,0,0,0,1,1
7372,Obligated,CML,5938(217),Stanislaus County,2016-02-17,2016-02-17,2016-02-17,2016-02-17,2016-02-23,-2915,...,0.00,Large,0,0,0,0,0,1,0,1
8506,Obligated,CML,5938(232),Stanislaus,2015-08-04,2015-08-04,2015-08-12,2015-08-12,2015-08-13,71494,...,0.00,Large,0,0,0,0,0,0,1,1
9103,Obligated,CML,5938(217),Stanislaus,2015-06-02,2015-06-02,2015-06-03,2015-06-08,2015-06-09,110000,...,0.00,Large,0,0,0,0,0,1,0,1
10284,Obligated,STPL,6089(060),Stan Area Aog,2014-12-30,2014-12-30,2014-12-30,2014-12-30,2014-12-30,0,...,0.00,Large,0,0,0,0,0,0,0,0
11731,Obligated,CML,6089(062),Stan Area Aog,2014-06-04,2014-06-04,2014-06-10,2014-06-13,2014-06-16,168207,...,0.00,Large,0,0,0,0,0,0,1,1
12739,Obligated,STPL,6089(056),Stan Area Aog,2014-02-18,2014-02-18,2014-02-18,2014-02-18,2014-02-20,0,...,0.00,Large,0,0,0,0,0,0,0,0
14101,Obligated,STPL,6089(079),Stanislaus Council Of Governments,2019-03-11,2019-03-21,2019-03-22,2019-03-27,2019-03-28,60000,...,0.00,Large,0,0,0,0,0,0,0,0
16633,Obligated,STPLNI,6089(079),Stanislaus Council Of Governments,2020-05-12,2020-05-12,2020-05-12,2020-05-12,2020-05-14,0,...,0.00,Large,0,0,0,0,0,0,0,0


* Agencies have no locodes listed under some obligations. 
* Agency name "Stanislaus" appears to be "Stanislaus County"

#### Agency Name 'YRTS'

In [55]:
df>>filter(_.agency=='Yrts')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
8716,Obligated,CML,6305(011),Yrts,2015-07-09,2015-07-13,2015-07-15,2015-07-17,2015-07-17,39283,...,0.00,Medium,0,1,0,0,0,0,0,1
10605,Obligated,CML,6305(008),Yrts,2014-10-13,2014-10-21,2014-10-23,2014-10-24,2014-10-24,45717,...,0.00,Medium,0,1,0,0,0,0,0,1
11204,Obligated,CML,6305(010),Yrts,2014-07-25,2014-07-25,2014-07-29,2014-07-30,2014-07-30,85000,...,0.00,Medium,0,1,0,0,0,0,0,1
11672,Obligated,CML,6305(007),Yrts,2014-06-19,2014-06-19,2014-06-19,2014-06-19,2014-06-20,0,...,0.00,Medium,0,0,0,0,0,0,0,0


In [56]:
df>>filter(_.agency=='Yrts')>>count(_.locode)

,locode,n
0,nan,4


In [57]:
df>>filter(_.agency=='Yrts')>>count(_.project_no)

,project_no,n
0,6305(007),1
1,6305(008),1
2,6305(010),1
3,6305(011),1


In [58]:
df>>filter(_.agency.str.contains('Yosemite'))>>count(_.locode)

,locode,n
0,6305,14


* 'Yrts' is actually ['Yarts'](https://yarts.com/), or Yosemite Area Regional Transportation System 

* no locodes listed, but if we use the first four numbers of the `project_no`, we get the same locode number as the `agency` Yosemite AND the official locode listed
* changing this in the "change_agency_to_locodelist" and uploading updated data in GCS

### How many agencies for a project location? 

In [59]:
#using _dla_utils
(_dla_utils.get_nunique(df, 'agency', 'project_location'))>>filter(_.n>2)

,project_location,n
9895,NaN,91
2945,FTA transfer,16
9106,Various Locations,13
2055,Citywide,8
9260,Various Locations Throughout The City,6
8976,Various,5
2331,Countywide,4
5872,Merced County,4
8397,Stanislaus County,4
5032,Interstate 5 From S/o Avenida Pico To S/o Aven...,3


In [60]:
df>>filter(_.project_location.str.contains("Interstate 5 From S/o Avenida Pico To S/o Aven"))

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
4349,Obligated,STPLN,6212(016),Caltrans,2017-04-11,2017-04-11,2017-05-05,2017-05-08,2017-05-22,0,...,0.00,Large,0,0,0,0,1,0,0,1
4656,Obligated,STPLN,6212(016),Caltrans,2016-12-05,2017-03-13,2017-03-22,2017-03-27,2017-04-07,1053000,...,0.00,Large,0,0,0,0,1,0,0,1
11202,Obligated,CMLN,6071(071),Ora Co Trans Au,2014-07-30,2014-07-30,2014-07-30,2014-07-30,2014-07-30,0,...,0.00,Large,0,0,0,0,1,0,0,1
16804,Obligated,CMLN,6071(071),Orange County Transportation Authority,2020-06-25,2020-06-25,2020-06-25,2020-06-25,2020-06-26,100450000,...,0.00,Large,0,0,0,0,1,0,0,1


In [61]:
df>>filter(_.project_location.str.contains("Merced County"))>>group_by(_.agency)>>count(_.project_no, _.locode)

,agency,project_no,locode,n
0,Jpam,6308(013),nan,1
1,Jpam,6308(014),nan,1
2,Jpam,6308(016),nan,1
3,Merced,5939(094),nan,1
4,Merced County,5939(093),5939,1
5,Merced County,7500(273),5939,2
6,Merced County,7500(278),5939,3
7,Merced County Association Of Governments,6095(061),6095,1
8,Merced County Association Of Governments,6095(074),6095,3
9,Merced County Association Of Governments,6095(077),6095,2


In [62]:
#using dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("Merced County"))), 'project_no','agency')

,agency,n
5,Transit Joint Powers Authority For Merced County,7
3,Merced County Association Of Governments,4
0,Jpam,3
2,Merced County,3
4,San Joaquin Council Of Governments,2
1,Merced,1


In [63]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("San Francisco Bay Area"))), 'project_no','agency')

,agency,n
1,Metropolitan Transportation Commission,45
0,Bay Area Air Quality Management District,3
2,Metropolitan Transportation Commission - Safe,3


In [64]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location.str.contains("San Joaquin County"))), 'project_no','agency')

,agency,n
5,San Joaquin County,6
4,San Joaquin Council Of Governments,2
0,Caltrans,1
1,Sacramento County,1
2,San Joaquin,1
3,San Joaquin Cog,1


* `agency` name 'San Joaquin' has same locode as 'San Joaquin County'

In [65]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location=="Various Locations (see Comments)")), 'project_no','agency')

,agency,n
2,Placer County,2
0,Imperial,1
1,Imperial County,1


#### Checking whats going on in Stanislaus

In [66]:
#using _dla_utils
_dla_utils.get_nunique((df>>filter(_.project_location=="Stanislaus County")), 'project_no','agency')

,agency,n
2,Stanislaus Council Of Governments,4
0,Stan Area Aog,3
1,Stanislaus,2
3,Stanislaus County,2


* multiple `agency` names for Stanislaus County and based on the first four digits of the `project_no` we have only two locodes
    *  Stanislaus County and Stanislaus looks to be the same
   

* despite the 49 entries under Stanislaus, there are no locodes, meaning they are listed as NaN. 
* checking to see if others agencies have NaN

In [67]:
len(df>>filter(_.agency=='Stanislaus'))

49

In [68]:
df>>filter(_.agency=='Stanislaus')>>count(_.locode)

,locode,n
0,nan,49


## Non-MPO Agencies
Looking at 'Non-MPO', 'ER NONE' and 'NaN' (or null)

#### Is Null

In [69]:
df>>filter(_.mpo.isnull())

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
22067,DISTRICT,HSIPL,7504(001),Caltrans,NaT,NaT,NaT,NaT,NaT,351200,...,NaN,Large,0,0,0,1,0,0,0,1
24900,DISTRICT,ATPL,5441(069),Moreno Valley,2021-10-25,NaT,NaT,NaT,NaT,0,...,0.00,Large,1,0,0,0,0,0,0,1
24909,DISTRICT,BPMPL,5073(077),Orange,2023-04-11,NaT,NaT,NaT,NaT,1474661,...,0.00,Large,0,0,1,0,0,0,0,1
25303,HQ,FTACML,6478(010),Tahoetrandist,2023-07-06,2023-07-06,2023-07-06,NaT,NaT,1000000,...,0.00,Medium,0,0,0,0,0,1,0,1
25304,HQ,FTACML,6343(007),Mctd,2023-07-27,2023-07-27,2023-08-01,NaT,NaT,1600000,...,0.00,Small,0,1,0,0,0,1,0,2
25305,HQ,FTASTPL,6156(010),Cen Cc Trans Au,2023-08-10,2023-08-10,NaT,NaT,NaT,400000,...,0.00,Small,0,1,0,0,0,0,0,1
25306,HQ,FTAFBPL,6003(061),Goldengate Hitd,2023-08-09,2023-08-09,NaT,NaT,NaT,2538414,...,0.00,Large,0,1,0,0,0,0,0,1
25307,HQ,CASB,6190(033),Usda Forest Ser,2017-12-14,2017-12-14,NaT,NaT,NaT,0,...,0.00,Small,0,0,0,0,0,1,0,1
25308,HQ,FTACML,5953(794),Los Angeles,2023-06-27,2023-06-27,2023-07-05,NaT,NaT,48416,...,0.00,Large,0,0,0,0,0,0,0,0
25309,HQ,FTASTPL,6312(032),Acsr,2023-06-15,2023-06-15,2023-06-21,NaT,NaT,80000000,...,0.00,Medium,0,0,0,0,0,1,0,1


### is ER NONE

In [70]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='ER NONE')))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Ridgecrest,2,primary_agency_name
1,California City,1,primary_agency_name
2,Camarillo,1,primary_agency_name
3,Chino Hills,1,primary_agency_name
4,Coachella,1,primary_agency_name
5,El Centro,1,primary_agency_name
6,Los Angeles County,1,primary_agency_name
7,Mission Viejo,1,primary_agency_name
8,Palm Springs,1,primary_agency_name
9,Pico Rivera,1,primary_agency_name


In [71]:
df>>filter(_.mpo=='ER NONE')>>filter(_.agency=='Ridgecrest')

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10222,Obligated,STPL,5385(043),Ridgecrest,2015-01-02,2015-01-02,2015-01-02,2015-01-02,2015-01-05,-8826,...,0.00,Large,0,0,0,1,0,1,0,2
10249,Obligated,STPL,5385(044),Ridgecrest,2014-12-31,2014-12-31,2014-12-31,2014-12-31,2015-01-04,-25490,...,0.00,Large,0,0,0,1,0,1,0,2


In [72]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='ER NONE')))>>filter(_.variable=='dist')

,value,count,variable
0,7,5,dist
1,8,3,dist
2,9,3,dist
3,11,2,dist
4,12,2,dist
5,10,1,dist


In [73]:
df>>filter(_.mpo=='ER NONE')>>filter(_.dist==8)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,adjusted_ac_requested,obligation_cat,active_transp,transit,bridge,street,freeway,infra_resiliency_er,congestion_relief,work_categories
10216,Obligated,RPMSTP,5282(018),Palm Springs,2014-12-09,2014-12-22,2015-01-05,2015-01-05,2015-01-05,0,...,0.00,Large,0,0,1,1,0,0,0,2
10218,Obligated,SRTSL,5294(009),Coachella,2015-01-02,2015-01-02,2015-01-02,2015-01-02,2015-01-05,0,...,0.00,Medium,1,0,0,1,0,0,0,2
10226,Obligated,SRTSL,5467(007),Chino Hills,2014-10-30,2014-12-19,2015-01-02,2015-01-02,2015-01-05,764130,...,0.00,Medium,1,0,0,1,0,1,0,3


#### Is Non-MPO

In [74]:
#using _dla_utils
(_dla_utils.find_top(df>>filter(_.mpo=='NON-MPO')))>>filter(_.variable=='primary_agency_name')

,value,count,variable
0,Humboldt County,898,primary_agency_name
1,Mendocino County,300,primary_agency_name
2,Lake County,163,primary_agency_name
3,Nevada County,152,primary_agency_name
4,Calaveras County,135,primary_agency_name
5,Tehama County,116,primary_agency_name
6,Trinity County,107,primary_agency_name
7,Caltrans,90,primary_agency_name
8,Plumas County,85,primary_agency_name
9,Tuolumne County,80,primary_agency_name
